In [4]:
import chess.pgn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import lichess.api
from lichess.format import PGN
import json
from itertools import islice
from efficient_apriori import apriori

ModuleNotFoundError: No module named 'efficient_apriori'

In [1]:
# Read chess games data file and create dataframe for games and a dictionary with users and their unique openings played
pgn = open("lichess_db_standard_rated_2016-05.pgn")

users = {}

games_arr = []
for x in range(6000000):
    headers = chess.pgn.read_headers(pgn)
    opening = headers.get('Opening').split(':')[0]
    opening = opening.split(' #')[0]
    if headers.get('Event') == "Rated Classical game":
        row = [headers.get('Event'), headers.get('White'), headers.get('Black'), opening, headers.get('WhiteElo'), headers.get('BlackElo')]
        games_arr.append(row)
        
        
        if users.get(headers.get('White')):
            users[headers.get('White')].append(opening)
        else:
            users[headers.get('White')] = [opening]
        
        if users.get(headers.get('Black')):
            users[headers.get('Black')].append(opening)
        else:
            users[headers.get('Black')] = [opening]
            
# Export users to json to avoid having to reread data
with open('users.json','w') as data: 
    json.dump(users, data)

FileNotFoundError: [Errno 2] No such file or directory: 'lichess_db_standard_rated_2016-05.pgn'

In [23]:
# Print average basket size
basket_size_sum = 0
count = 0
for k in users:
    count += 1
    basket_size_sum += len(users[k])
    
print(basket_size_sum/count)

39.62776699798044


In [2]:
# Export games dataframe to csv to avoid having to reread data
dfGames = pd.DataFrame(games_arr, columns = ['Event', 'White', 'Black', 'Opening', 'WhiteElo', 'BlackElo'])
print(len(games_arr))
dfGames.to_csv('out.csv', encoding='utf-8', index=False)

NameError: name 'pd' is not defined

In [42]:

with open('gambits filtered/users.json') as f:
    data = f.read()
    
idkdict = json.loads(data)
for item in islice(idkdict, 1):
    print(idkdict[item])

['Owen Defense', "King's Knight Opening", 'Philidor Defense', 'Ruy Lopez', 'Ruy Lopez', 'Philidor Defense', 'Russian Game', "Van't Kruijs Opening", 'Italian Game', 'Scotch Game', 'Zukertort Opening', 'Alekhine Defense', 'Caro-Kann Defense', "Queen's Pawn Game", 'Three Knights Opening', 'Italian Game', 'Ruy Lopez', "Bishop's Opening", 'Italian Game', 'Sicilian Defense', 'Philidor Defense', 'Italian Game', 'Philidor Defense', "King's Pawn Game", "King's Pawn Game", 'Italian Game', "Van't Kruijs Opening", 'Sicilian Defense', "Queen's Gambit Accepted", 'Scotch Game', 'French Defense', 'Philidor Defense', 'Italian Game', 'Ruy Lopez', 'French Defense', 'Vienna Game', 'Italian Game', 'Italian Game', 'Italian Game', 'French Defense', "Queen's Pawn Game", 'Sicilian Defense', "King's Pawn Game", "Queen's Gambit Accepted", 'Reti Opening', 'Four Knights Game', 'Scotch Game', "King's Pawn Game", 'Czech Defense', 'Sicilian Defense', 'Scotch Game', 'Ruy Lopez', 'Russian Game', "Queen's Gambit Accepte

In [43]:
df = pd.read_csv('gambits filtered/out.csv')
print(df)

                        Event        White             Black  \
0        Rated Classical game    edumenoyo   platito_de_lefa   
1        Rated Classical game     a7md2010           dreypat   
2        Rated Classical game     raking41          mariofil   
3        Rated Classical game     JOHN14-6            naikus   
4        Rated Classical game      guevara          Sacchina   
...                       ...          ...               ...   
1501075  Rated Classical game  skailain279  pincheweylopezXD   
1501076  Rated Classical game     Zimerman         paramirez   
1501077  Rated Classical game       Airon8           ardbano   
1501078  Rated Classical game  Danieloaiza         dannyatac   
1501079  Rated Classical game  papadesarah         photomium   

                        Opening  WhiteElo  BlackElo  BlackRatingDiff  \
0                  Owen Defense      1770      1625             -7.0   
1             Hungarian Opening      1704      1764            -12.0   
2              

In [44]:
openingswithusers = [(k, v) for k, v in idkdict.items()]
openings = []
for i in range(len(openingswithusers)):
    openings.append(openingswithusers[i][1])

In [45]:
itemsets, rules = apriori(openings, min_support = 0.15, min_confidence = 0.85)

In [54]:
results = []
for i in range(5):
    rules_rhs = filter(lambda rule: len(rule.lhs) == (1 + i) and len(rule.rhs) == 1, rules)
    for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
        results.append(rule)

In [55]:
for j in range(len(results)):
    print("{}\n".format(results[j])) # sorted by lift

{English Opening, French Defense} -> {Sicilian Defense} (conf: 0.853, supp: 0.180, lift: 1.723, conv: 3.436)

{French Defense, Scandinavian Defense} -> {Sicilian Defense} (conf: 0.858, supp: 0.219, lift: 1.733, conv: 3.550)

{French Defense, Philidor Defense} -> {Sicilian Defense} (conf: 0.859, supp: 0.211, lift: 1.736, conv: 3.583)

{Bishop's Opening, French Defense} -> {Sicilian Defense} (conf: 0.860, supp: 0.163, lift: 1.737, conv: 3.601)

{Caro-Kann Defense, Queen's Pawn Game} -> {Sicilian Defense} (conf: 0.863, supp: 0.151, lift: 1.744, conv: 3.695)

{Italian Game, Philidor Defense} -> {Sicilian Defense} (conf: 0.865, supp: 0.174, lift: 1.748, conv: 3.752)

{French Defense, Italian Game} -> {Sicilian Defense} (conf: 0.869, supp: 0.189, lift: 1.755, conv: 3.843)

{French Defense, Nimzowitsch Defense} -> {Sicilian Defense} (conf: 0.869, supp: 0.157, lift: 1.756, conv: 3.867)

{French Defense, Ruy Lopez} -> {Sicilian Defense} (conf: 0.877, supp: 0.173, lift: 1.771, conv: 4.091)

{Phi

In [68]:
allopenings = df["Opening"].unique()
winrate = dict.fromkeys(allopenings, 0)
frequency = dict.fromkeys(allopenings, 0)
for index, row in df.iterrows():
    temp = row["Opening"]
    if temp in winrate:
        if "Defense" in temp:
            if row["BlackRatingDiff"] > 0:
                winrate[temp] = winrate[temp] + 1          
        else:
            if row["WhiteRatingDiff"] > 0:
                winrate[temp] = winrate[temp] + 1
        frequency[temp] = frequency[temp] + 1

In [69]:
for key in frequency:
    if frequency.get(key, 0) < 30:
        winrate.pop(key)
    else:
        winrate[key] = (winrate[key] / frequency[key])
        winrate[key] = round(winrate[key], 3)

In [75]:
print(winrate["Queen's Pawn Game"])

0.501


In [74]:
print(frequency["Queen's Pawn Game"])

106256
